# CANCER DETECTION IN DERMOSCOPIC IMAGES

## Introduction

[According to the World Health Organization](https://www.who.int/news-room/fact-sheets/detail/cancer), skin cancer is the fifth most common type of cancer in the world, with 1.2 million new cases diagnosed in 2020. 

Thanks to the pattern recognition capabilities of Convolutional Neural Networks, melanoma can be early detected by visually analyzing the characteristic of a skin mole. In this notebook we propose a cancer detection algorithm based on some of the most typical CNN arquitectures (Resnet, Inception, MobileNet, DenseNet and a from scrath network). 

We used ISIC DB 2018 dataset to evaluate the experiments. A highly imbalanced dataset containing 2750 dermoscopic images, publicly available here: https://challenge.isic-archive.com/data

The dataset was stored in my personal Google Drive account.

*This code was developed from scratch out of personal curiosity, so there may be some minor errors and hard-coding*. 


## Development

### Import libraries

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import argparse
import collections
import re

from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torch.autograd import Function
from torch.autograd import Variable
import torch.cuda.amp as amp

import statistics as stats
import random
import scipy.io
from PIL import Image
import cv2 as cv


### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.chdir('/content/drive/My Drive/') #Mount Drive 

Mounted at /content/drive


In [ ]:
#Use GPU if available. (Runtime -> Change runtime type -> GPU)
use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if use_gpu else "cpu")

In [ ]:
%load_ext tensorboard

import tensorflow as tf
import datetime, os
from torch.utils.tensorboard import SummaryWriter

logdir = "runs/exp1"
writer = SummaryWriter(log_dir=logdir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Auxiliary functions

In [ ]:
def lesion(mask):
  """
    This function generates a binary mask that separates background pixels (healthy skin) and the lesion.
    The mask is used to eliminate the maximum amount of background pixels when reading an image from the dataset.
  
      input: mask(h,w), indices[0,6]: Bg, Others, Cysts...
      output: mask'(h,w), indices[0,1]: Bg, Fg 
  """

  new = np.zeros(mask.shape)
  new[(mask==0)] = 0 
  new[(mask==1)] = 1 
  new[(mask==2)] = 1  
  new[(mask==3)] = 1  
  new[(mask==4)] = 1 
  new[(mask==5)] = 1 
  new[(mask==6)] = 1 

  return new

In [ ]:
def get_segment_crop(img,tol=0, mask=None):
  """
  This function crops and image given a background-foreground binary mask. 
    inputs: image [c, h, w]
             mask [1, h, w]
    output: image [c, h', w'], where h'w' < hw        
  """
  if mask is None:
    mask = img > tol
  
  return img[np.ix_(mask.any(1), mask.any(0))]

### Preprocessing

In [ ]:
#ISIC DB mean/std 
mean = [0.1769, 0.1479, 0.1367]
std = [0.0352, 0.0378, 0.0417]

#These operations are applied to the training set. 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize([300,]),
     transforms.RandomCrop(size=(300)),
     #transforms.RandomRotation(90),
     transforms.RandomVerticalFlip(p=0.5), 
     transforms.RandomHorizontalFlip(p=0.5)
    ])

#These operations are applied to the validation and test set to evaluate the model.
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize([300,], interpolation=transforms.InterpolationMode.NEAREST),
     transforms.RandomCrop(size=300)
    ])

#Data augmentation in the RGB channels
data_aug = transforms.Compose(
    [transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
     transforms.ToTensor(),
     transforms.Normalize(mean=mean, std=std),
     transforms.ToPILImage()
     
    ])

### Dataset and Dataloaders

In [ ]:
class db_isic_Dataset(torch.utils.data.Dataset):
  def __init__(self, root, idx, transform=None, data_aug=None, subset='train'):
    self.root = root
    self.idx = idx
    self.db = idx['db']
    self.transform = transform
    self.data_aug = data_aug
    self.subset = subset
    
    subset_idx = []
    if(self.subset == 'train'):
      subset_idx = np.where(self.db['set']==1) #Train
    elif(self.subset == 'val'):
      subset_idx = np.where(self.db['set']==2) #Val 
    elif(self.subset == 'test'):
      subset_idx = np.where(self.db['set']==3) #Test


    self.db = self.db[subset_idx]
    self.imPath, self.gt, self.dlabel = self.db['imPath'], self.db['gt'], self.db['dlabel']
    self.imagePath = ['']*len(self.imPath)
    self.gtPath = ['']*len(self.imPath)
   
  def __getitem__(self, index):
    imagePath = os.path.join(self.root, self.imPath[index][0])
    gtPath = os.path.join(self.root, self.gt[index][0])

    image_t = Image.open(imagePath)

    mask = scipy.io.loadmat(gtPath)['smgt'] #Segmentation ground truth mask
    foreground = lesion(mask) #Get foreground mask

    image_t = get_segment_crop(np.asarray(image_t), mask=foreground) #Eliminate background pixels

    dlabel = self.dlabel[index][0] #Diagnostic label. '0': non-cancer, '1': cancer
    dlabel= torch.from_numpy(dlabel)
    dlabel = np.clip(dlabel, 0, 1)
    dlabel = dlabel.float()
    
    if self.data_aug: #Data augmentation
      image_t = self.data_aug(image_t)
      
    if self.transform: #Geometrical transformations. 
      image_t = self.transform(image_t)

    return image_t, dlabel

  def __len__(self):
    return len(self.imPath)

In [ ]:
ROOT = "/content/drive/My Drive/db_isic/"
path = "/content/drive/My Drive/db_isic/idx/isic_2017.mat"
idx = scipy.io.loadmat(path)

trainset = db_isic_Dataset(root=ROOT, idx=idx, transform=transform, subset='train')
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

validationset = db_isic_Dataset(root=ROOT, idx=idx, transform=transform, subset='val')
validationloader = torch.utils.data.DataLoader(validationset, batch_size=4, shuffle=True)


data_loaders = {"train": trainloader, "val": validationloader}

### Arquitectures

#### Resnet101

In [ ]:
def resnet101(num_classes=1):
  model = models.resnet101(pretrained=True)
  model.fc = nn.Sequential(
      nn.Linear(2048, num_classes),
      nn.Sigmoid()
  )

  return model

#### Resnet18

In [ ]:
def resnet18(num_classes=1):
  model = models.resnet18(pretrained=True)

  model.fc = nn.Sequential(
      nn.Linear(512, num_classes),
      nn.Sigmoid()
  )

  return model  

#### Inception v3

In [ ]:
def inceptionv3(num_classes=1):
  model = models.inception_v3(pretrained=True)

  model.fc = nn.Sequential(
      nn.Linear(2048, num_classes),
      nn.Sigmoid()
  )

  return model #Forward function returns the actual output in output[0]

#### Mobilenet v3

In [ ]:
def mobilenet(num_classes=1):
  model = models.mobilenet_v3_large(pretrained=True)

  model.classifier = nn.Sequential(
      nn.Linear(960, num_classes),
      nn.Sigmoid()
  )
  return model

#### Densenet169

In [ ]:
def densenet169(num_classes=1):
  model = models.densenet169(pretrained=True)

  model.classifier = nn.Sequential(
      nn.Linear(1664, num_classes),
      nn.Sigmoid()
  )

  return model

#### From scratch

In [ ]:
class scratch(torch.nn.Module):

  def __init__(self):  
        super().__init__()
        input_channels = 3
        output_channels = 1
        
        #DOWNSAMPLING ---------------------------------------------------------
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(input_channels, 64, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU()       
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128, 128, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU()      
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU()
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 512, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU()
        )
        self.layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(512, 1024, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.BatchNorm2d(1024),
            torch.nn.ReLU(),
            torch.nn.Conv2d(1024, 1024, kernel_size=(3,3), stride=1, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.BatchNorm2d(1024),
            torch.nn.ReLU()
        )
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.fc = nn.Sequential(
            nn.Linear(1024, output_channels),
            nn.Sigmoid()
        )

  def forward(self, x):

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x) 
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)

    print(x.shape)
    x = self.fc(x)

    return x

        

In [ ]:
#Debug
model = scratch()

hw = 300
input  = torch.rand((2,3,hw,hw)) #Generate random 3-channel tensor (input image)

out = model(input)

print(out.shape) #(2, 1)
print(out) #Scores

In [ ]:
def model_selector(model="resnet101"):
  if(model=="resnet101"):
    return resnet101()
  elif(model=="resnet18"):
    return resnet18()
  elif(model=="inception"):
    return inception()
  elif(model=="mobilenet"):
    return mobilenet()
  elif(model=="densenet"):
    return densenet169()
  elif(model=="scratch"):
    return scratch()  
  else:
    print("The model you selected is not available.")
    print("...Resnet101 is selected.") 
    return resnet101()         

### Loss Function

*To be completed...*

NOTE: the idea is to penalize False Negatives as they have a major impact than False Positives. **We do not want to miss detecting cancer**. We can use Tversky Loss to balance the contribution of FP and FN.  

In [ ]:
class TverskyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(TverskyLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=0.7, beta=0.3):
	"""
        inputs: raw output predictions (without activation). Shape: [batch, num_classes, h, w]
        targets: ground truth labels (one-hot encoded). Shape: [batch, num_classes, h, w]
	alpha: False Positives weighting coefficient
	beta: False Negatives weighting coefficient
        """
        
        inputs = torch.nn.functional.softmax(inputs, dim=1)


        #flatten label and prediction tensors
        inputs = inputs.contiguous().view(-1)
        targets = targets.contiguous().view(-1)
        
        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()    
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()

                

        Tversky = (2*TP + smooth) / (2*TP + alpha*FP + beta*FN + smooth)  

   
        return 1 - Tversky

### Model and Parameters

In [ ]:
#Arquitectures available: "resnet101", "resnet18", "inception", "mobilenet", "densenet", "scratch"
arquitecture = "resnet101"
model = model_selector(model=arquitecture)

model.to(device)

criterion =  nn.BCELoss() #Binary Cross Entropy*. *We need to calculate the probabilities a priori of cancer vs non-cancer.
#criterion = nn.MSELoss() #Mean Squared Error
#criterion = TverskyLoss() #Tversky Loss

#Layer parameters for Resnet models*. 
p1 = [p for p in model.layer2.parameters() if p.requires_grad]
p2 = [p for p in model.layer3.parameters() if p.requires_grad]
p3 = [p for p in model.layer4.parameters() if p.requires_grad]


#Stochastic Gradient Descend with momentum. 
optimizer = optim.SGD([{'params': p1, 'lr': 1e-7},   
                       {'params': p2, 'lr': 1e-6},
                       {'params': p3, 'lr': 1e-4}
                        ], lr=1e-3 , momentum=0.999, nesterov=True)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.95)

num_epochs = 50

## Training

In [ ]:
checkpoint_path = "/content/drive/My Drive/Checkpoints/diagnostic.tar" #Load and save model each epoch in a .tar file.

saveCheckpoint = False #Saves the model
loadCheckpoint = False #Loads the model
#---------------------------------------------------------------------------------
train_loss = []
validation_loss = []


if(loadCheckpoint):
  print("Loading checkpoint... | Path: ", checkpoint_path)
  checkpoint = torch.load(checkpoint_path, map_location=device)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict']) 


for epoch in range(num_epochs):
    optimizer.zero_grad()
    if(epoch > 0):  
      if(saveCheckpoint): #Save model
          print("Saving checkpoint... | epoch: ", epoch, "| Path: ", checkpoint_path)
          torch.save({'model_state_dict': model.state_dict(),  
                      'optimizer_state_dict': optimizer.state_dict(),
                      'scheduler_state': scheduler.state_dict()},
                      checkpoint_path)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:

        if phase == 'train':
            model.train()  # Set model to training mode
            print("Epoch: ", epoch, "| Training phase |")
        else:
            model.eval()  # Set model to evaluate mode
            model.apply(bn_eval)
            print("Epoch: ", epoch, "| Validation phase |")

        for i, data in enumerate(data_loaders[phase],0):
          inputs, labels = data  
          #labels = labels.long() 
                  
          inputs, labels = inputs.to(device), labels.to(device)

          #Predict output probabilities
          y_score = model(inputs)          

          #Calculate Loss
          loss = criterion(y_score, labels)
          
          if phase == 'train':            
            optimizer.zero_grad()
            loss.backward() #Backpropagation
            optimizer.step()  
            scheduler.step() #Sch step
            train_loss.append(loss.item())
            writer.add_scalar("Loss/train", loss.item(), i) #Tensorboard


          if phase == 'val':
            validation_loss.append(loss.item())
            writer.add_scalar("Loss/val", loss.item(), i) #Tensorboard


          #if(i%5 ==0):
          #  print("Scores: ", y_score)
          #  print("Labels: ", labels)
          
writer.flush()
 
    

## Launch Tensorboard

*It won't open if Third-party cookies are blocked.

In [ ]:
%tensorboard --logdir=runs

In [ ]:
writer.close()